### Layers : Fundamental blocks of Neural Network

In [1]:
import torch
from torch.nn import Linear, ReLU
import torch.nn as nn
import numpy as np
from torch.autograd import Variable



### Linear(in_features,out_features,bias)

定義 輸入和輸出層的維度(weight的維度)

In [2]:
myLayer = Linear(in_features=10,out_features=5,bias=True)
inp = Variable(torch.randn(1,10))
myLayer = Linear(in_features=10,out_features=5,bias=True) 
myLayer(inp)

tensor([[ 1.3674, -0.1240, -0.1245, -0.5646,  1.0261]],
       grad_fn=<AddmmBackward>)

In [3]:
myLayer.weight

Parameter containing:
tensor([[ 0.1086, -0.0032, -0.1726, -0.2917, -0.2321, -0.1221,  0.0398, -0.2243,
         -0.3114, -0.2054],
        [ 0.0961,  0.0321,  0.1991, -0.1609, -0.2867, -0.1709, -0.2107,  0.2972,
          0.0900, -0.2611],
        [-0.3053,  0.1653, -0.0303, -0.2112, -0.1097, -0.1235,  0.2295, -0.0543,
          0.2380, -0.0214],
        [ 0.1242, -0.1319,  0.0426, -0.2045,  0.1032,  0.2621, -0.0854,  0.2583,
          0.0328, -0.0680],
        [-0.1783,  0.0088, -0.1117, -0.0364, -0.3002, -0.2273, -0.2780, -0.2134,
         -0.0474,  0.2673]], requires_grad=True)

In [4]:
myLayer.bias

Parameter containing:
tensor([ 0.0237, -0.2723, -0.1328,  0.1506,  0.1731], requires_grad=True)

### Stacking Linear layers

建立多層神經層透過迭代

In [ ]:
myLayer1 = Linear(10,5)
myLayer2 = Linear(5,2)
myLayer2(myLayer1(inp))

tensor([[0.2186, 0.3087]], grad_fn=<AddmmBackward>)

### PyTorch Non-linear Activations

使用兩種呼叫激勵函數的不同方式

In [ ]:
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
myRelu = ReLU()
myRelu(sample_data)


tensor([[1., 2., 0., 0.]])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
f = F.relu(sample_data) # Much simpler.
f

tensor([[1., 2., 0., 0.]])

### Neural Network 

利用類別建立多層神經網路

In [ ]:
class MyFirstNetwork(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyFirstNetwork,self).__init__() 
        self.layer1 = nn.Linear(input_size,hidden_size) 
        self.layer2 = nn.Linear(hidden_size,output_size)
    def layer(self):
        first  = [self.layer1.weight.data,self.layer1.bias.data]
        second = [self.layer2.weight.data,self.layer2.bias.data]
        return first,second
    def forward(self,input): 
        out = self.layer1(input) 
        out = F.relu(out)
        out = self.layer2(out) 
        out = F.softmax(out,dim=1)
        return out

model = MyFirstNetwork(224*224,56*56,2)
print(model)

MyFirstNetwork(
  (layer1): Linear(in_features=50176, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)


### Loss

介紹兩種不同的loss function

In [ ]:
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()

分類問題常用的loss function: 交叉熵(cross-entropy)

In [ ]:
def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1 - prediction)

### loss(x,class)=−x[class]+log(∑exp(x[j]))

利用上述公式自訂函數與內建函數比較

In [ ]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.LongTensor(3).random_(5)) 
output = loss(input, target)
output.backward()
print(output.data.item())

def CrossEntropyLoss(input,target):
    sum = 0
    for i in range(len(input)):
        tmp = 0
        for j in range(len(input[0])):
            tmp+=np.e**(input[i][j])
        sum+=-input[i][target[i]]+np.log(tmp)
    return float(sum)/len(input)

print(CrossEntropyLoss(input.data.numpy(),target.data.numpy()))

1.659788966178894
1.659788995200539


### Optimizer

將多個圖形利用自訂的類別建立成一個物件

In [ ]:
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
# ### Optimizer
class DogsAndCatsDataset(Dataset):
    def __init__(self,root_dir,size=(224,224)):
        self.files = glob(root_dir)
        self.size = size
        self.arr = np.array([[0],[1]],int)
    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        input = Variable(torch.from_numpy(np.asarray(Image.open(self.files[idx]).resize((1,224*224)), float)))
        if(idx%2==0):
            target = Variable(torch.from_numpy(self.arr[0]))
        else:
            target = Variable(torch.from_numpy(self.arr[1])) 
        return input,target

dataset=DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/classifier/*.jpg")

### Training

In [ ]:
import torch.optim as optim
from time import perf_counter
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001,momentum=0.9)
print("Training......")

t1 = perf_counter()
t  = t1
for i in range(10):
    index = 0
    print("epoch",i)
    for input, target in dataset:
        data=input[:,:,0].view(1,-1).float()+input[:,:,1].view(1,-1).float()+input[:,:,2].view(1,-1).float()
        output = model(data/(3*255.))
        optimizer.zero_grad()
        loss = loss_fn(output, target.long())
        loss.backward()
        optimizer.step()
        index+=1
        if(perf_counter()-t > 30):
            t = perf_counter()
            print("  Completion ratio :",str(index*100.0/60)+"%")
t2  = perf_counter()

print(model)
print("--------------------------------------")
print("layer1 :","\nweight :\n",model.layer()[0][0].numpy(),"\nbias :\n",model.layer()[0][1].numpy())
print("")
print("layer2 :","\nweight :\n",model.layer()[1][0].numpy(),"\nbias :\n",model.layer()[1][1].numpy())
print("time :",str((t2-t1)/60.0)+" min")

print("\ntest.....")
test = DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/sample_test/*.jpg")
data1=test[0][0][:,:,0].view(1,-1).float()+test[0][0][:,:,1].view(1,-1).float()+test[0][0][:,:,2].view(1,-1).float()
data2=test[1][0][:,:,0].view(1,-1).float()+test[1][0][:,:,1].view(1,-1).float()+test[1][0][:,:,2].view(1,-1).float()
output1 = model(data1/(3*255.))
output2 = model(data2/(3*255.))
print(output1)
print(output2)

Training......
epoch 0
  Completion ratio : 50.0%
  Completion ratio : 98.33333333333333%
epoch 1
  Completion ratio : 46.666666666666664%
  Completion ratio : 95.0%
epoch 2
  Completion ratio : 43.333333333333336%
  Completion ratio : 91.66666666666667%
epoch 3
  Completion ratio : 40.0%
  Completion ratio : 88.33333333333333%
epoch 4
  Completion ratio : 36.666666666666664%
  Completion ratio : 86.66666666666667%
epoch 5
  Completion ratio : 35.0%
  Completion ratio : 83.33333333333333%
epoch 6
  Completion ratio : 31.666666666666668%
  Completion ratio : 81.66666666666667%
epoch 7
  Completion ratio : 35.0%
  Completion ratio : 86.66666666666667%
epoch 8
  Completion ratio : 40.0%
  Completion ratio : 90.0%
epoch 9
  Completion ratio : 40.0%
